# Image tiling for annotation

#### Meanings of arguments
- ```-ratioheight``` : proportion of tile  w.r.t height of image. Example 0.5 means dividing the image in two bands w.r.t height.
- ```-ratiowidth``` : proportion of tile w.r.t to width of image. Example 1.0 means the width of the tile is the same as the image.
- ```-overlapfactor``` : percentage of overlap. It should be less than 1.
- ```-rmheight``` : percentage of height to remove or crop at bottom and top
- ```-rmwidth``` : percentage of width to remove or crop on each side of the image
- ```-pattern``` : "**/*.JPG" will get all .JPG images in directory and subdirectories. On windows it will get both .JPG and .jpg. On unix it will only get .JPG images


In [5]:
# New script for tiling data
# images_to_tile = r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images"
# destination_directory = r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images-tiled"
!python ../../HerdNet/tools/patcher.py "D:\PhD\Data per camp\Wet season\Leopard rock\Camp 1_8+35-36\Rep 3" 0 0 0 -overlapfactor 0.1  -ratiowidth 0.5 -ratioheight 0.5 -rmheight 0 -rmwidth 0 -dest "D:\PhD\Data per camp\Wet season\Leopard rock\Camp 1_8+35-36\Rep 3 - tiled" -pattern "**/*.JPG"

c:\Users\Machine Learning\anaconda3\envs\label-backend\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.4 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()

Exporting patches: 100%|██████████| 126/126 [03:20<00:00,  1.59s/it]


# Pre-annotating data for Labelstudio

In [53]:
from dotenv import load_dotenv
load_dotenv('../.env')

from datalabeling.annotator import Annotator
import os
from pathlib import Path
import torch
from tqdm import tqdm

### Creating a JSON file to be uuploaded to Label studio

In [ ]:
# Example
# provide correct alias, "pt", "onnx"
alias = "last" # the aliases are found in mlflow tracker UI, use "last-1" to use the previous model
name = "obb-detector" # detector, "obb-detector"
handler = Annotator(mlflow_model_alias=alias,
                    mlflow_model_name=name,
                    is_yolo_obb= name.strip() == "obb-detector",
                    # dotenv_path="../.env"
                    )
path_img_dir=r"D:\PhD\Africa Parks\Liuwa aerial survey_ALL\CENSUS 2019\DAY 2 CENSUS 2019_CONVERTED\AP 2019 day 2 - tiled"
root="D:\\"
save_json_path = os.path.join(Path(path_img_dir).parent, f"{Path(path_img_dir).name}_preannotation_label-studio.json")

# build and saves json
directory_preds = handler.build_upload_json(path_img_dir=path_img_dir,
                                            root=root,
                                            save_json_path=save_json_path,
                                            pattern="**/*.JPG")

### Pre-annotating an existing project using Label studio API
It seems that it will not work well (i.e. filtering) with older projects created prior to Label studio software update.
It is the **recommended way of pre-annotating data in Labelstudio**.

In [4]:
# provide correct alias, "pt", "onnx"
aliases = ["version6", "version5"]
project_id = 88 # insert correct project_id by loooking at the url
for alias in aliases:
    name = "obb-detector" # detector, "obb-detector"
    handler = Annotator(mlflow_model_alias=alias,
                        mlflow_model_name=name,
                        confidence_threshold=0.25,
                        is_yolo_obb=name.strip() == "obb-detector",
                        dotenv_path="../.env")
    handler.upload_predictions(project_id=project_id)

02/14/2025 17:04:18 - INFO - datalabeling.annotator.models -   Computing device: cuda:0
Uploading predictions:   0%|          | 1/566 [00:05<50:45,  5.39s/it]

Prediction performed in 5.326715707778931 seconds.


Uploading predictions:   0%|          | 2/566 [00:05<22:09,  2.36s/it]

Prediction performed in 0.18864798545837402 seconds.


Uploading predictions:   1%|          | 3/566 [00:05<12:57,  1.38s/it]

...

Uploading predictions: 100%|█████████▉| 564/566 [01:27<00:00,  6.80it/s]

Prediction performed in 0.09375667572021484 seconds.
Prediction performed in 0.10938262939453125 seconds.


Uploading predictions: 100%|██████████| 566/566 [01:28<00:00,  6.66it/s]

Prediction performed in 0.11609530448913574 seconds.
Prediction performed in 0.11216449737548828 seconds.


Uploading predictions: 100%|██████████| 566/566 [01:28<00:00,  6.42it/s]


**Before running the script below, make sure that you have exported the annotations so you can revert back!!!**

In [59]:
#  Cleaning annotations - NO WAY BACK
name = "obb-detector"
handler = Annotator(mlflow_model_alias="version6",
                        mlflow_model_name=name,
                        confidence_threshold=0.25,
                        is_yolo_obb=name.strip() == "obb-detector",
                        dotenv_path="../.env")

# Select project
project_id = 88
project = handler.labelstudio_client.get_project(id=project_id)

# Delete annotations saved with label "wildlife" assigned by the predictor
tasks = project.get_tasks()
for task in tqdm(tasks,desc="correcting annotations"):
        task_id = task['id']
        img_url = task['data']['image']

        if len(task["annotations"][0]['result'])>1:
            results_to_keep = []
            annot_id = task["annotations"][0]["id"]
            for annot in task['annotations'][0]['result']:
                if annot['value']['rectanglelabels'][0] != 'wildlife':
                    results_to_keep.append(annot)
                    # print(annot['value'],annot['id'],end="\n")
            # print(f"Updating annotations {annot_id} from task {task_id}.")
            # print(results_to_keep)
            project.update_annotation(annot_id,result=results_to_keep)

02/15/2025 14:31:21 - INFO - datalabeling.annotator.models -   Computing device: cuda:0
correcting annotations: 100%|██████████| 566/566 [00:31<00:00, 18.12it/s]


In [55]:
len(task['annotations']), len(task['annotations'][0]['result']), task['id'], task["annotations"][0]["id"]

(1, 8, 152009, 60568)

In [52]:
task['annotations'][0]['result'][0] #['value']['rectanglelabels']

{'original_width': 4506,
 'original_height': 2730,
 'image_rotation': 0,
 'value': {'x': 86.51410749209387,
  'y': 43.134913182520606,
  'width': 1.667947510957612,
  'height': 2.548828125,
  'rotation': 0,
  'rectanglelabels': ['wildlife']},
 'id': 'LXtHpQTq6J',
 'from_name': 'label',
 'to_name': 'image',
 'type': 'rectanglelabels',
 'origin': 'prediction',
 'score': 0.24935585260391235}

In [54]:
results_to_keep = []
for annot in task['annotations'][0]['result']:
    if annot['value']['rectanglelabels'][0] != 'wildlife':
        results_to_keep.append(annot)
        print(annot['value'],annot['id'],end="\n")

In [48]:
results_to_keep

[]

In [ ]:
project.update_annotation(annotation_id=...,)

02/15/2025 13:51:57 - ERROR - label_studio_sdk._legacy.client -   
--------------------------------------------
Request URL: http://localhost:8080/api/annotations/LXtHpQTq6J
Response status code: 404
Response content:



<!doctype html>
<html lang="en">
<head>
  <title>Label Studio</title>
  <meta charset="utf-8">
  <meta name="author" content=""/>
  <meta name="viewport" content="width=device-width, initial-scale=0.9, shrink-to-fit=no">

  <!-- Address string color -->
  <meta name="theme-color" content="#272727"> <!-- Chrome, Firefox OS and Opera -->
  <meta name="msapplication-navbutton-color" content="#272727">  <!-- Windows Phone -->
  <meta name="apple-mobile-web-app-status-bar-style" content="#272727">  <!-- iOS Safari -->

  <link href="/static/images/favicon.8992b8aa4126.ico" rel="shortcut icon"/>
  <link href="/static/css/main.cf908d032736.css" rel="stylesheet"/>
  <link href="/react-app/main.css?v=d9b816" rel="stylesheet">

  
  <title>404 | Label Studio</title>
  <link rel=

HTTPError: 404 Client Error: Not Found for url: http://localhost:8080/api/annotations/LXtHpQTq6J

To speed up inference on intel, make changes inn ultralytics/nn/autobackend.py:
```
- device_name = "AUTO:NPU,GPU,CPU" # CPU, GPU, NPU, AUTO,"AUTO:GPU,NPU"
- inference_mode = "LATENCY" # OpenVINO inference modes are 'LATENCY', 'THROUGHPUT' (not recommended), or 'CUMULATIVE_THROUGHPUT'
- LOGGER.info(f"Using OpenVINO {inference_mode} mode for inference...")
- ov_compiled_model = core.compile_model(
                ov_model,
                device_name=device_name,  # AUTO selects best available device, do not modify
                config={"PERFORMANCE_HINT": inference_mode,
                        "CACHE_DIR": os.environ["OPENVINO_CACHE_MODEL"]}, # make sure to set environment variable
            )
```

In [ ]:
# using path_to_weights
# go to ultralytics.nn.autobackend to modify ov_compiled device to "AUTO:NPU,GPU,CPU"

use_sliding_window=True

handler = Annotator(path_to_weights=r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model",
                    is_yolo_obb=True,
                    tilesize=1280,
                    overlapratio=0.1,
                    use_sliding_window=use_sliding_window,
                    confidence_threshold=0.5,
                    device="NPU", # "cpu", "cuda"
                    tag_to_append=f"-sahi:{use_sliding_window}",
                    dotenv_path="../.env")

project_id = 3 # insert correct project_id by loooking at the url
top_n=10
handler.upload_predictions(project_id=project_id,top_n=top_n)

In [ ]:
from label_studio_ml.utils import get_local_path
from urllib.parse import unquote, quote
import os
path = unquote("/data/local-files/?d=savmap_dataset_v2%5Cimages_splits%5C003a34ee6b7841e6851b8fe511ebe102_0.JPG")
get_local_path(path,download_resources=False)#,os.path.exists(get_local_path(path))

# Inference with Sahi

In [ ]:
from ultralytics import YOLO
from PIL import Image
import time
import numpy as np
from datalabeling.annotator import Detector
from dotenv import load_dotenv

In [ ]:
# load env variable, loads model cache location!!
load_dotenv('../.env')

In [ ]:
IMAGE_PATH = r"D:\savmap_dataset_v2\images_splits\00a033fefe644429a1e0fcffe88f8b39_1.JPG"

## Optimizing with Openvino

To speed up inference on intel, make changes inn ultralytics/nn/autobackend.py:
```
- device_name = "AUTO:NPU,GPU,CPU" # CPU, GPU, NPU, AUTO,"AUTO:GPU,NPU"
- inference_mode = "LATENCY" # OpenVINO inference modes are 'LATENCY', 'THROUGHPUT' (not recommended), or 'CUMULATIVE_THROUGHPUT'
- LOGGER.info(f"Using OpenVINO {inference_mode} mode for inference...")
- ov_compiled_model = core.compile_model(
                ov_model,
                device_name=device_name,  # AUTO selects best available device, do not modify
                config={"PERFORMANCE_HINT": inference_mode,
                        "CACHE_DIR": os.environ["OPENVINO_CACHE_MODEL"]}, # make sure to set environment variable
            )
```

In [ ]:
# Define detector
# to speed up inference on intel, make
model = Detector(path_to_weights=r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model",
                confidence_threshold=0.1,
                overlap_ratio=0.1,
                tilesize=1280,
                device='CPU',
                use_sliding_window=False,
                is_yolo_obb=True)

In [ ]:
image = Image.open(IMAGE_PATH)

while True:
    start_time = time.perf_counter()
    print(model.predict(image,return_coco=True,nms_iou=0.5))
    end_time = time.perf_counter()
    print(f"Device took {end_time-start_time:.2f} seconds.")

    break

In [ ]:
# inference with openvino
import openvino as ov
import openvino.properties.hint as hints
import torch
import torchvision.transforms as F
from ultralytics.utils import DEFAULT_CFG
from ultralytics.cfg import get_cfg
from ultralytics.data.converter import coco80_to_coco91_class

# load validator
args = get_cfg(cfg=DEFAULT_CFG)
det_model = YOLO(r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best.pt")
det_validator = det_model.task_map[det_model.task]["validator"](args=args)
det_validator.is_coco = True
det_validator.class_map = coco80_to_coco91_class()
det_validator.names = det_model.model.names
det_validator.metrics.names = det_validator.names
det_validator.nc = det_model.model.model[-1].nc
det_validator.stride = 32
args = get_cfg(cfg=DEFAULT_CFG)
det_model = YOLO(r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best.pt")

core = ov.Core()
det_model_path = r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model\best.xml"
det_ov_model = core.read_model(det_model_path)

device = "AUTO:NPU,GPU" # CPU, NPU, GPU "AUTO:NPU,GPU,CPU" 

print("Available core devices: ",core.available_devices)

# reshaping for batch prediction
input_layer = det_ov_model.input(0)
output_layer = det_ov_model.output(0)
new_shape = ov.PartialShape([1, 3, 1280, 1280])
det_ov_model.reshape({input_layer.any_name: new_shape})

ov_config = {hints.performance_mode: hints.PerformanceMode.THROUGHPUT,
             "CACHE_DIR": '../models/model_cache'}

if ("GPU" in core.available_devices) and device=="GPU":
    ov_config["GPU_DISABLE_WINOGRAD_CONVOLUTION"] = "YES"
det_compiled_model = core.compile_model(det_ov_model, device, ov_config)

def infer(image):
    image = det_validator.preprocess({"img":image,"batch_idx":torch.Tensor([0]),
                                      "cls":torch.Tensor([0]),
                                      "bboxes":torch.Tensor([0.,0.,0.,0.])})["img"]
    results = det_compiled_model(image)
    preds = torch.from_numpy(results[det_compiled_model.output(0)])
    return det_validator.postprocess(preds) #torch.from_numpy(result[0])

In [ ]:
# image = Image.open(IMAGE_PATH)
# image = F.PILToTensor()(image)[None,:,:1280,:1280]
# infer(image)

In [ ]:
# inference with pt
# model = YOLO(r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best.pt",task='obb')

In [ ]:
# rescaling input images
# model(image/255.)

In [ ]:
# inference with openvino
# model_vino = YOLO(r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model",task='obb')
# model_vino(image/255.)

In [ ]:
# sahi_model_obb = Detector(path_to_weights=r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model",
#                     confidence_threshold=0.6,
#                     overlap_ratio=0.1,
#                     tilesize=640,
#                     is_yolo_obb=True)

In [ ]:
# image_path = r"D:\savmap_dataset_v2\images\0d1ba3c424ad4414ac37dbd0c93460ea.JPG"
# image = Image.open(image_path)
# print(image.size)

In [ ]:
# result = sahi_model_obb.predict(image,False)

In [ ]:
# result
# result.export_visuals('../.tmp')

## Sahi inference calibration

In [ ]:
from itertools import product

In [ ]:
# hyperparams
overlap_ratios = [0.1,0.2,0.3]
tilesizes = [640,2*640,3*640]
imgsz = [640,2*640,3*640]

for ratio, tilesize, image_size in product(overlap_ratios,tilesizes,imgsz):
    print(ratio,tilesize,image_size)
    # Define detector
    # to speed up inference on intel, make
    model = Detector(path_to_weights=r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model",
                    confidence_threshold=0.1,
                    overlap_ratio=0.1,
                    tilesize=2000,
                    imgsz=1280,
                    device='CPU',
                    use_sliding_window=True,
                    is_yolo_obb=True)
    
    #TODO


# YOLO data_config.yaml 

In [ ]:
import yaml
import json
from arguments import Arguments

In [ ]:
# load yaml
with open(r"D:\PhD\Data per camp\IdentificationDataset\data_config.yaml",'r') as file:
    yolo_config = yaml.load(file,Loader=yaml.FullLoader)
yolo_config

In [ ]:
# load label mapping
args = Arguments()
with open(r"D:\PhD\Data per camp\IdentificationDataset\label_mapping.json",'r') as file:
    label_map = json.load(file)
names = [p['name'] for p in label_map if p['name'] not in args.discard_labels ]
label_map = dict(zip(range(len(names)),names))
label_map

In [ ]:
yolo_config.update({'names':label_map,'nc':len(label_map)})
yolo_config

In [ ]:
with open(r"D:\PhD\Data per camp\IdentificationDataset\data_config.yaml",'w') as file:
    yaml.dump(yolo_config,file,default_flow_style=False, sort_keys=False)

# Dataset distribution

## Visualize distribution per annotation project

In [ ]:
from datalabeling.dataset import convert_json_annotations_to_coco, load_coco_annotations
from pathlib import Path
import json
import pandas as pd
from collections import Counter
from dotenv import load_dotenv
from label_studio_sdk import Client
# from itertools import chain
import os

In [2]:
dotenv_path=r"..\.env"
load_dotenv(dotenv_path=dotenv_path)
# Connect to the Label Studio API and check the connection
LABEL_STUDIO_URL = os.getenv('LABEL_STUDIO_URL')
API_KEY = os.getenv("LABEL_STUDIO_API_KEY")
labelstudio_client = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

In [7]:
project_id = 69 # project id
project = labelstudio_client.get_project(id=project_id)

# 
num_images = dict()

# Iterating 
tasks = project.get_tasks()
annotator_id = 0
labels = []
for task in tasks:
    result = task['annotations'][0]['result']
    img_labels = []
    for annot in result:
        img_labels = annot['value']['rectanglelabels'] + img_labels
    labels = labels + img_labels
    # update stats holder
    for label in set(img_labels):
        try:
            num_images[label] += 1
        except:
            num_images[label] = 1

stats = {f"{k}":labels.count(k) for k in set(labels)}
print("Number of instances for each label is:\n",stats,end="\n\n")
print("Number of images for each label is:\n",num_images)


Number of instances for each label is:
 {'impala': 5, 'wildebeest': 568, 'zebra': 1045, 'reedbuck': 1, 'lechwe': 392, 'buffalo': 397, 'wildlife': 3}

Number of images for each label is:
 {'zebra': 93, 'buffalo': 18, 'wildlife': 3, 'reedbuck': 1, 'impala': 3, 'lechwe': 26, 'wildebeest': 44}


In [ ]:
ls_dir = r"D:\PhD\Data per camp\Exported annotations and labels\Wet season - Rep 1\all\labelstudio"
dest_dir = Path(ls_dir).with_name("coco-format")
save_excel_path = Path(ls_dir).with_name("stats.xlsx")

# Uncomment to run if needed
# convert_json_annotations_to_coco(input_dir=ls_dir,
#                                  dest_dir_coco=str(dest_dir),
#                                  ls_client=labelstudio_client,
#                                  parse_ls_config=True)

In [ ]:
coco_annotations_dict = load_coco_annotations(dest_dir)
coco_annotations_dict

In [ ]:
def get_labels_count(coco_annotation:dict):

    result = Counter([annot['category_id'] for annot in coco_annotation['annotations']])

    label_map = {cat['id']:cat['name'] for cat in coco_annotation['categories']}

    result = {label_map[k]:v for k,v in result.items()}

    return result

label_stats = dict()

for img_dir,coco_path in coco_annotations_dict.items():

    with open(coco_path,'r') as f:
        coco_annotation = json.load(fp=f)
    
    label_stats[img_dir] = get_labels_count(coco_annotation)

label_stats = pd.DataFrame.from_dict(label_stats,orient='index').fillna(0)

In [ ]:
label_stats

In [ ]:
# uncomment to save
label_stats.to_excel(save_excel_path)

## Visualize splits' distribution

In [ ]:
import yaml
import pandas as pd
import os
from pathlib import Path

In [ ]:
# load yaml
with open(r"D:\PhD\Data per camp\Extra training data\WAID\data_config.yaml",'r') as file:
    yolo_config = yaml.load(file,Loader=yaml.FullLoader)
yolo_config

In [ ]:
label_map = yolo_config['names']

In [ ]:
split = 'train'

path_dataset = os.path.join(yolo_config['path'],yolo_config[split][0])
path_dataset = path_dataset.replace('images','labels')

path_dataset

In [ ]:
labels = list()

for txtfile in Path(path_dataset).glob("*.txt"):

    df = pd.read_csv(txtfile,sep=" ",names = ['class','x','y','w','h'] )
    df['class'] = df['class'].astype(int)    
    df['image'] = txtfile.stem
    labels.append(df)


In [ ]:
df = pd.concat(labels,axis=0)
df['class'] = df['class'].map(label_map)

In [ ]:
images_per_class = dict()
for cls in df['class'].unique():
    num_imge = df.loc[df['class'] == cls,'image'].unique().shape[0]
    images_per_class[cls] = num_imge

In [ ]:
print("Split:", split)
print(images_per_class)

In [ ]:
print('Split:',split)
print(df['class'].value_counts())

In [ ]:
df['class'].value_counts().plot(kind='bar',figsize=(10,5),logy=True,title=f"{split} label distribution")

# Computing metrics on Validation set

In [ ]:
from ultralytics import YOLO
# from pathlib import Path
import torch

In [ ]:
# Load a model
path = r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\base_models_weights\yolov8-wildai-obb.pt"
# path = r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\base_models_weights\yolov5su.pt"
model = YOLO(path)  

In [ ]:
pred = model.predict(r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\images\01f1653a94f14044bf11d78c5b4221d1.JPG")

In [ ]:
[result.obb for result in pred]

In [ ]:
pred[0].obb.xyxy

In [ ]:
pred[0].obb.cls

In [ ]:
pred[0].obb.conf

In [ ]:
# Customize validation settings
validation_results = model.val(data=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\data_config.yaml",
                                imgsz=640,
                                batch=8,
                                conf=0.25,
                                iou=0.5,
                                device="cpu")

In [ ]:
# Compute predictions
from datalabeling.annotator import Detector

handler = Detector(path_to_weights=path,confidence_threshold=0.3)
predictions = handler.predict_directory(r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\images")

# Dataset label format conversion

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def check_label_format(loaded_df:pd.DataFrame)->str:
    """checks label format

    Args:
        loaded_df (pd.DataFrame): target values

    Raises:
        NotImplementedError: when the format is not yolo or yolo-obb

    Returns:
        str: yolo or yolo-obb
    """

    num_features = len(loaded_df.columns)

    if num_features == 5:
        return "yolo"
    elif num_features == 9:
        return "yolo-obb"
    else:
        raise NotImplementedError(f"The number of features ({num_features}) in the label file is wrong. Check yolo or yolo-obb format.")

In [ ]:
label_path = r"D:\PhD\Data per camp\DetectionDataset\Rep 1\train\labels\DJI_20231002150401_0009_0_48_0_1271_640_1911.txt"
df = pd.read_csv(label_path,sep=' ',header=None)
df

In [ ]:
isinstance(df.iloc[:,0].dtype, np.dtypes.IntDType)

In [ ]:
check_label_format(df)

In [ ]:
len(df.columns)

In [ ]:
df.columns = ['id','x1','y1','x2','y2','x3','y3','x4','y4']

df